In [1]:
import gurobipy as gb

teams = ['UConn', 'Creighton', 'DePaul', 'Xavier', 'St. Johns', 'Marquette', 'Providence', 'Georgetown', 'Butler', 'Villanova', 'Seton Hall']
num_teams = len(teams)
slots = 22

model = gb.Model()
model.update()

# [home team, away team, slot number]
x = model.addVars(num_teams, num_teams, slots, vtype=gb.GRB.BINARY, name="game")

# Defining tuple for each pair of teams
# Two tuples for each team with home/away and away/home
distance_pairs = [ (i,j) for i in range(num_teams) for j in range(i+1, num_teams) ]

# Model variable representing distance pair and slot
# "Slot" value in this variable will represent the distance between two matchups
# i.e. if team 0 plays team 2 in slots 1 and 4, d[0, 2, 3] = 1
d = model.addVars(distance_pairs, slots, vtype=gb.GRB.BINARY, name="game")

for (i,j) in distance_pairs:
    for k in range(slots):
        for l in range(k+1,slots):
            model.addConstr( x[i,j,k] + x[j,i,l] + (1-d[i,j,l-k]) <= 2 )
            model.addConstr( x[j,i,k] + x[i,j,l] + (1-d[i,j,l-k]) <= 2 )

# As we raise k, we are essentially raising the distance between two matchups.
# Maximizing for k*d[i, j, k] maximizes the sum of time between all matchup pairs.
model.setObjective(
    gb.quicksum(
        k*d[i,j,k] for (i,j) in distance_pairs for k in range(slots)
    ),
    gb.GRB.MAXIMIZE
)

# Each team must play each other team at home once and only once
# This constraint forces that each team place each team twice
for i in range(num_teams):
    for j in range(num_teams):
        if i != j:
            # Each Team Plays every team in conference at home once and only once
            model.addConstr(
                gb.quicksum(x[i, j, k] for k in range(slots)) == 1, 
                name='1home1away')
        if i == j:
            # Nobody Plays Themselves
            model.addConstr(
                gb.quicksum(x[i, j, k] for k in range(slots)) == 0, 
                name='CantPlaySelf')


#A team cannot play twice in one slot
for k in range(slots):
    for t in range(num_teams):
        model.addConstr(
            gb.quicksum(x[t, j, k] for j in range(num_teams)) + 
            gb.quicksum(x[i, t, k] for i in range(num_teams)) <= 1,
            name="OncePerSlot")
        

#Teams cannot play only at home or away first two games, last two games
for i in range(num_teams):
    # Team cannot play at home in slot 0 and 1
    model.addConstr(
        gb.quicksum(x[i, j, k] for j in range(num_teams) for k in range(2)) <= 1, 
        name='NoFirstTwoHome')
for j in range(num_teams):
    # Team cannot play away in slot 0 and 1
    model.addConstr(
        gb.quicksum(x[i, j, k] for i in range(num_teams) for k in range(2)) <= 1, 
        name='NoFirstTwoAway')

for i in range(num_teams):
    # Team cannot play at home in last two slots
    model.addConstr(
        gb.quicksum(x[i, j, k] for j in range(num_teams) for k in range(slots - 2, slots)) <= 1, 
        name='NoLastTwoHome')
for j in range(num_teams):
    # Team cannot play away in last two slots
    model.addConstr(
        gb.quicksum(x[i, j, k] for i in range(num_teams) for k in range(slots - 2, slots)) <= 1, 
        name='NoLastTwoAway')


#Teams cannot play at home or away first two weekends, last two weekends
for i in range(num_teams):
    # Team cannot play at home in slot 0 and 2
    model.addConstr(
        gb.quicksum(x[i, j, k] for j in range(num_teams) for k in range(0, 3, 2)) <= 1, 
        name='NoFirstTwoWkndHome')
for j in range(num_teams):
    # Team cannot play away in slot 0 and 2
    model.addConstr(
        gb.quicksum(x[i, j, k] for i in range(num_teams) for k in range(0, 3, 2)) <= 1, 
        name='NoFirstTwoWkndAway')

for i in range(num_teams):
    # Team cannot play at home in slot (slots - 1) and in slot (slots - 3) (last two weekends)
    model.addConstr(
        gb.quicksum(x[i, j, k] for j in range(num_teams) for k in range(slots-3, slots, 2)) <= 1, 
        name='NoLastTwoWkndHome')
for j in range(num_teams):
    # Team cannot play away in slot (slots - 1) and in slot (slots - 3) (last two weekends)
    model.addConstr(gb.quicksum(x[i, j, k] for i in range(num_teams) for k in range(slots-3, slots, 2)) <= 1,
                     name='NoLastTwoWkndAway')


# Teams must have one bye in each half of the season
for t in range(num_teams):
    # First Half Bye
    model.addConstr(
        gb.quicksum(x[t, j, k] for j in range(num_teams) for k in range(int(slots/2))) + 
        gb.quicksum(x[i, t, k] for i in range(num_teams) for k in range(int(slots/2))) <= int(slots/2) - 1,
        name="FirstHalfBye")
    #Second Half Bye
    model.addConstr(
        gb.quicksum(x[t, j, k] for j in range(num_teams) for k in range(int(slots/2), slots)) + 
        gb.quicksum(x[i, t, k] for i in range(num_teams) for k in range(int(slots/2), slots)) <= int(slots/2) - 1,
        name="FirstHalfBye")
    
# Teams must play at home two out of first five weekends
for i in range(num_teams):
    model.addConstr(
        gb.quicksum(x[i, j, k] for j in range(num_teams) for k in range(0, 10, 2)) <= 5,
        name='TwoHomeFirstFiveWknd'
    )

# No more than Two Home Games Consecutively
for i in range(num_teams):
    for k in range(2, slots):
        # No more than two home games consecutively
        model.addConstr(
            gb.quicksum(x[i, j, k-2] + x[i, j, k-1] + x[i, j, k] for j in range(num_teams)) <= 2,
            name='NoThreeConsecHome')

# No more than Two Away Games Consecutively
for j in range(num_teams):
    for k in range(2, slots):
        # No more than two home games consecutively
        model.addConstr(
            gb.quicksum(x[i, j, k-2] + x[i, j, k-1] + x[i, j, k] for i in range(num_teams)) <= 2,
            name='NoThreeConsecHome')       


    


model.optimize()

if model.Status == gb.GRB.OPTIMAL:
    for k in range(slots):
        print("Slot ", k, ": ")
        for i in range(num_teams):
            for j in range(num_teams):
                if x[i, j, k].X >= 0.9:
                    print(teams[j], " @ ", teams[i])

# The code below was provided by ChatGPT                    
elif model.Status == gb.GRB.INFEASIBLE:
    
    print("Model is infeasible. Computing IIS...")
    model.computeIIS()
    model.write("infeasible.ilp")  # Save the infeasibility report to a file
else:
    print(f"Optimization was stopped with status {model.Status}")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-29
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12650H, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 26334 rows, 3872 columns and 106117 nonzeros
Model fingerprint: 0x26061683
Variable types: 0 continuous, 3872 integer (3872 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 25476 rows and 1452 columns
Presolve time: 0.10s
Presolved: 858 rows, 2420 columns, 25740 nonzeros
Variable types: 0 continuous, 2420 integer (2420 binary)

Root relaxation: objective 1.270500e+04, 1341 iterations, 0.13 seconds (0.12 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |